##### Load data

In [35]:
import pandas as pd
import json

df = pd.read_excel('../data/Elsevier-LIS/Texts.xlsx')
links = df['Pii'].tolist()
abs = df['Abstract'].tolist()
hts = df['Highlights'].tolist()

link_to_keywords = {}
with open('../data/Elsevier-LIS/Keywords.json', 'r') as f:
    link_to_keywords = json.load(f)

In [36]:
keywords = []

for i, link in enumerate(links):
    try:
        keywords.append(link_to_keywords[link])
    except:
        keywords.append([])

In [3]:
ahts, hats = [], []

for i, link in enumerate(links):
    ahts.append(abs[i] + ' ' + hts[i])
    hats.append(hts[i] + ' ' + abs[i])

keywords = list(link_to_keywords.values())

##### GPT-3.5

In [6]:
# api-key: sk-LUmyq6sPQ5g0Wm362329654c68Aa493aAe738018C4190661
# api-base: 'https://api.aimd5.com/v1'
# api-key: sk-CwIP4r9DuGSeoEpB4d1a9e762655470e8096F3Db0fB789E2
# api-base: 'https://scornful-kit-abcd1256.koyeb.app'

In [4]:
from openai import OpenAI
import os

client = OpenAI(
   api_key = 'sk-f2Ojy8n1PhauagN53971475f4e10456288Ad91544d5c3a15',
   base_url = 'https://burn.hair/v1' 
)

In [ ]:
keywords_pred = []

In [8]:
from tqdm import tqdm

start_pos = len(keywords_pred)
for i, text in enumerate(tqdm(hts[start_pos:])):
    completion = client.chat.completions.create(
        model = 'gpt-3.5-turbo-1106',
        messages = [
            {"role": "system", "content": "You are an expert in the academic field of computer science"},
            {"role": "user", "content": "Based on the text below, give the keyword set of the paper in which it is located, sort the results from high to low importance, and separate the results with commas. Note that only the results are given without any additional explanations."+text}
        ]            
    )
    
    keyword_pred = completion.choices[0].message.content
    keywords_pred.append(keyword_pred)

100%|██████████| 1442/1442 [47:56<00:00,  1.99s/it]  


In [9]:
new_df = pd.DataFrame()
new_df['Keywords'] = keywords_pred
new_df.to_excel('LLM_Pred/Elsevier-CS-GPT-H.xlsx', index=False)

##### Claude3

In [26]:
# api-key: sk-LUmyq6sPQ5g0Wm362329654c68Aa493aAe738018C4190661
# api-base: https://api.v3.cm
from openai import OpenAI
import openai
import os

openai.api_key = 'sk-bvnuW6ca0HXnslYBF4De808c61Dd4241847bA21603051231'
openai.base_url = 'https://api.v3.cm/v1/'
openai.default_headers = {
    "x-foo":"true"
}

In [4]:
keywords_pred = []

In [33]:
len(keywords_pred)

2996

In [32]:
from tqdm import tqdm

pos = len(keywords_pred)
for i, text in enumerate(tqdm(abs[pos:])):
    completion = openai.chat.completions.create(
        model = 'claude-3-sonnet-20240229',
        messages = [
            {"role": "system", "content": "You are an expert in the academic field of computer science"},
            {"role": "user", "content": "Based on the text below, give the keyword set of the paper in which it is located, sort the results from high to low importance, and separate the results with commas. Note that only the results are given without any additional explanations."+text}
        ]            
    )

    keyword_pred = completion.choices[0].message.content
    keywords_pred.append(keyword_pred)

100%|██████████| 680/680 [33:45<00:00,  2.98s/it]  


In [34]:
new_df = pd.DataFrame()
new_df['Keywords'] = keywords_pred
new_df.to_excel('LLM_Pred/Elsevier-CS-Claude-A.xlsx', index=False)

##### Evaluation

In [53]:
import re

new_df = pd.read_excel('LLM_Pred/Elsevier-LIS-GPT-AH.xlsx')
raw_pred_keywords = new_df['Keywords'].tolist()
pred_keywords = []
regex = r'\d. '
for elem in raw_pred_keywords:
    keyword = []
    if ':' in elem:
        pos = elem.rindex(':')
        elem = elem[pos+1:]
        if ',' in elem:
            keyword = elem.split(',')
        elif '\n' in elem:
            keyword = elem.split('\n')
    if ':' not in elem:
        if ',' in elem:
            keyword = elem.split(',')
        elif '\n' in elem:
            keyword = elem.split('\n')

    new_keyword = []
    for i, word in enumerate(keyword):
        word = word.strip()
        if len(word.split(' ')) >=5:
            continue
        if '- ' in word:
            pos = word.index(' ')
            word = word[pos+1:]
        elif re.search(regex, word):
            pos = re.search(regex, word).span()[1]
            word = word[pos:]

        if word != '':
            if word[-1] == '.':
                new_keyword.append(word[:-1])
            else:
                new_keyword.append(word)

    keyword = [word.strip() for word in new_keyword]
    pred_keywords.append(keyword)

In [7]:
import nltk

porter = nltk.PorterStemmer()

def stemmer(raw_sequences):
    stemmed_sequences = []

    for i, words in enumerate(raw_sequences):
        new_words = []
        for word in words:
            if type(word) == list:
                # for h_candidates and a_candidates
                word = word[0]
            items = word.split()
            new_word = ' '.join(porter.stem(item) for item in items)
            new_words.append(new_word.strip())
        stemmed_sequences.append(new_words)
    
    return stemmed_sequences

In [38]:
import logging

class Logger(object):

    def __init__(self, filename, level='info'):
        level = logging.INFO if level == 'info' else logging.DEBUG
        self.logger = logging.getLogger(filename)
        self.logger.propagate = False
        self.logger.setLevel(level)  #

        th = logging.FileHandler(filename, 'a')

        self.logger.addHandler(th)

log = Logger('LLM_Pred/Elsevier-LIS.log')

In [9]:
def calPRF(num_c, num_e, num_s):
    F1 = 0.0
    P = float(num_c) / float(num_e) if num_e!=0 else 0.0
    R = float(num_c) / float(num_s) if num_s!=0 else 0.0
    if (P + R == 0.0):
        F1 = 0
    else:
        F1 = 2 * P * R / (P + R)
    return P, R, F1

def getPRF(references, predictions, log):
    num_c_5, num_c_10, num_c_15 = 0, 0, 0
    num_e_5, num_e_10, num_e_15 = 0, 0, 0
    num_s = 0
    for i  in range(len(references)):
        reference = references[i]
        prediction = predictions[i]
        j = 0
        for candidate in prediction[:15]:
            if candidate in reference:
                if j<5:
                    num_c_5 += 1
                    num_c_10 += 1
                    num_c_15 += 1
                elif (j<10 and j>=5):
                    num_c_10 += 1
                    num_c_15 += 1
                elif (j<15 and j>=10):
                    num_c_15 += 1
            j += 1
        
        if len(prediction[0:5]) == 5:
            num_e_5 += 5
        else:
            num_e_5 += len(prediction[0:5])
        
        if len(prediction[0:10]) == 10:
            num_e_10 += 10
        else:
            num_e_10 += len(prediction[0:10])
        
        if len(prediction[0:15]) == 15:
            num_e_15 += 15
        else:
            num_e_15 += len(prediction[0:15])

        num_s += len(reference)
    
    P, R, F1 = calPRF(num_c_5, num_e_5, num_s)
    log.logger.info("P@5:{} R@5:{} F1@5:{}".format(P,R,F1))
    P, R, F1 = calPRF(num_c_10, num_e_10, num_s)
    log.logger.info("P@10:{} R@10:{} F1@10:{}".format(P,R,F1))
    P, R, F1 = calPRF(num_c_15, num_e_15, num_s)
    log.logger.info("P@15:{} R@15:{} F1@15:{}".format(P,R,F1))

In [54]:
gold_standards_stem = stemmer(keywords)
pred_candidates_stem = stemmer(pred_keywords)

In [55]:
log.logger.info('GPT-AH')
getPRF(gold_standards_stem, pred_candidates_stem, log)